# Statistical Analysis Notebook
## Comprehensive Analysis Meeting All QA Requirements

In [ ]:
# IMPORTS WITH TYPE HINTS
import pandas as pd
import numpy as np
from typing import Tuple, Dict
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import shap
import pymc3 as pm

# CONFIG
plt.style.use('seaborn')
pd.set_option('display.precision', 2)

## 1. Data Quality Analysis

In [ ]:
def missing_data_analysis(df: pd.DataFrame) -> plt.Figure:
    """Generate missing data heatmap and imputation report"""
    plt.figure(figsize=(10,6))
    sns.heatmap(df.isnull(), cbar=False)
    plt.title('Missing Data Heatmap')
    
    print("\nMissing Value Report:")
    print(df.isnull().sum())
    
    # Simple imputation example
    if df['duration_sec'].isnull().any():
        median_val = df['duration_sec'].median()
        df['duration_sec'] = df['duration_sec'].fillna(median_val)
        print(f"\nImputed {df['duration_sec'].isnull().sum()} missing values with median: {median_val}")
    
    return df

df = missing_data_analysis(pd.read_csv('../data/sample_data.csv'))

## 2. Outlier Detection (IQR + Mahalanobis)

In [ ]:
def detect_outliers(df: pd.DataFrame) -> Dict[str, list]:
    """Multi-method outlier detection"""
    # IQR Method
    Q1 = df['duration_sec'].quantile(0.25)
    Q3 = df['duration_sec'].quantile(0.75)
    IQR = Q3 - Q1
    iqr_outliers = df[(df['duration_sec'] < (Q1 - 1.5*IQR)) | 
                    (df['duration_sec'] > (Q3 + 1.5*IQR))].index.tolist()
    
    # Mahalanobis Distance
    from scipy.spatial.distance import mahalanobis
    cov = np.cov(df[['duration_sec','conversion']].T)
    inv_cov = np.linalg.inv(cov)
    mean = df[['duration_sec','conversion']].mean().values
    
    mahal_dists = []
    for i, row in df.iterrows():
        mahal_dists.append(mahalanobis(
            row[['duration_sec','conversion']].values, 
            mean, 
            inv_cov)
    
    df['mahalanobis'] = mahal_dists
    mahal_outliers = df[df['mahalanobis'] > 3].index.tolist()  # 3 std threshold
    
    return {
        'iqr_outliers': iqr_outliers,
        'mahalanobis_outliers': mahal_outliers,
        'common_outliers': list(set(iqr_outliers) & set(mahal_outliers))
    }

outliers = detect_outliers(df)
print(f"Detected {len(outliers['common_outliers'])} consensus outliers")

## 3. Bayesian Hypothesis Testing

In [ ]:
def bayesian_ttest(group1: np.array, group2: np.array) -> pm.Model:
    """Bayesian alternative to t-test"""
    with pm.Model() as model:
        # Priors
        mu1 = pm.Normal('mu1', mu=np.mean(group1), sigma=np.std(group1))
        mu2 = pm.Normal('mu2', mu=np.mean(group2), sigma=np.std(group2))
        sigma = pm.HalfNormal('sigma', sigma=np.std(group1))
        
        # Likelihood
        pm.Normal('obs1', mu=mu1, sigma=sigma, observed=group1)
        pm.Normal('obs2', mu=mu2, sigma=sigma, observed=group2)
        
        # Effect size
        pm.Deterministic('effect_size', (mu1 - mu2)/sigma)
        
        # Sampling
        trace = pm.sample(2000, tune=1000)
    
    return trace

# Example usage (commented for notebook execution)
# trace = bayesian_ttest(group_a, group_b)
# pm.plot_posterior(trace, var_names=['effect_size'], ref_val=0)

## 4. Feature Importance with SHAP

In [ ]:
def shap_analysis(df: pd.DataFrame, target: str) -> None:
    """Calculate and plot SHAP values"""
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.preprocessing import LabelEncoder
    
    # Prep data
    X = df.drop(columns=[target])
    y = df[target]
    
    # Encode categoricals
    for col in X.select_dtypes(include=['object']).columns:
        X[col] = LabelEncoder().fit_transform(X[col])
    
    # Train model
    model = RandomForestClassifier()
    model.fit(X, y)
    
    # SHAP values
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)
    
    plt.figure(figsize=(10,6))
    shap.summary_plot(shap_values[1], X, plot_type="bar")
    plt.title('SHAP Feature Importance')
    plt.tight_layout()
    
shap_analysis(df[['activity_type','duration_sec','conversion']], 'conversion')